In [1]:
import pathlib
# from tqdm import tqdm
# from tqdm.notebook import trange, tqdm
from tqdm.autonotebook import tqdm
import random
import collections
import math 
import shutil
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal as signal
from time import time
import platform
from skimage.util.shape import view_as_windows
from scipy import ndimage
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%matplotlib inline
plt.rcParams.update({'figure.max_open_warning': 0})
np.set_printoptions(precision=4, suppress=True)
pd.set_option('display.float_format', '{:,.4f}'.format)

_platform = platform.platform()
print('platform:', _platform, platform.node())
if 'Linux' in _platform: # linux
    video_path = '/home/cclee/image_data/'
    out_path = '/home/cclee/tmp/rat/'
    train_path = '/home/cclee/tmp/rat/training_grooming/'
elif 'Darwin' in _platform: # MAC OS X
    mp4_path = '/Users/cclee/image_data/ratmp4/'
    groom_path = '/Users/cclee/rat_data/training_grooming/'
    data_path = '/Users/cclee/rat_data/' 
    out_path = '/Users/cclee/rat_data/new_out/' 
    new_groom_path = '/Users/cclee/rat_data/new_grooming/'
    new_groom_path_old = '/Users/cclee/rat_data/new_grooming_old/'
    short_path = '/Users/cclee/rat_data/short_clip/'
elif 'Windows' in _platform: # Windows
    if platform.node()=='Mozart':
        mp4_path = 'e:/image_data/ratmp4'
        groom_path = 'e:/rat_data/training_grooming'
        data_path = 'e:/rat_data/'
        out_path = 'e:/rat_data/new_out'
        new_groom_path = 'e:/rat_data/new_grooming/'
        new_groom_path_old = 'e:/rat_data/new_grooming_old/'
        short_path = 'e:/rat_data/short_clip/'
    else:
        mp4_path = 'd:/image_data/ratmp4'
        groom_path = 'd:/rat_data/training_grooming'
        data_path = 'd:/rat_data/'  
        out_path = 'd:/rat_data/new_out'
        new_groom_path = 'd:/rat_data/new_grooming/'
        new_groom_path_old = 'd:/rat_data/new_grooming_old/'
        short_path = 'd:/rat_data/short_clip/'
    
dpath = pathlib.Path(data_path)
mp4path = pathlib.Path(mp4_path)
outpath = pathlib.Path(out_path)
groompath = pathlib.Path(groom_path)
newgroompath = pathlib.Path(new_groom_path)
shortpath = pathlib.Path(short_path)
newgroom_oldpath = pathlib.Path(new_groom_path_old)

read_cols = (0, 6, 10, 11, 12)
exp_lst = [921111,921216,930203,930217, 930302, 930309]

if not outpath.exists():
    outpath.mkdir()    
    
if not newgroompath.exists():
    newgroompath.mkdir() 
    
if not shortpath.exists():
    shortpath.mkdir()     

<ipython-input-1-527f99a2bbab>:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


platform: Windows-10-10.0.18362-SP0 zephyrus


In [22]:
fontFace = cv2.FONT_HERSHEY_SIMPLEX
_black = (0, 0, 0) 
_white = (255, 255, 255)
font_color = (0,255,0)

def stitch_video(mp4_lst, prefix):
    video_file = str(mp4_lst[0])
    cap = cv2.VideoCapture(video_file)
    bOpenVideo = cap.isOpened()
    if bOpenVideo == False:
        print('Open Video failed ', mp4_lst[0])
        return

    video_fps = cap.get(cv2.CAP_PROP_FPS) *2
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    outName = '{}/{:s}.mp4'.format(str(dpath), prefix) 
    vidw = cv2.VideoWriter(outName, cv2.VideoWriter_fourcc(*'avc1'),  ####XVID'), 
                           video_fps, (width, height), True)  # Make a video

    if vidw.isOpened()==True:
        pbar = tqdm(total=len(mp4_lst))
        for clip in mp4_lst:
            cap = cv2.VideoCapture(str(clip))
            bOpenVideo = cap.isOpened()
            if bOpenVideo == False:
                print('Open Video failed ', clip)
                return
            title_str = clip.stem.split('_')[1]
            while True:
                bVideoRead, frame = cap.read()
                if bVideoRead:
                    cv2.putText(frame, title_str, (5,frame.shape[0]-10), fontFace, 0.5, font_color)
                    vidw.write(frame)
                else:                                        
                    break
            pbar.update(1)
            
        vidw.release()  
        print('out video: ', outName)
    else:
        print('ERROR: output video, ' , outName)
    
    print('stitch_video finished')

In [25]:
df = pd.read_csv('misclassified.csv', index_col= 0)
print(df)
df['clip_name'] = df['clip'].apply(lambda x: x.split('_',1)[-1])
df['grooming'] = df['clip'].apply(lambda x: x[0]=='G')
display(df)

df_mis_groom_FN = df[df['grooming']==True]
mis_groom_FN_lst = list(df_mis_groom_FN['clip_name'])
print('mis_groom_FN_lst', len(mis_groom_FN_lst))

df_mis_nongroom_FP = df[df['grooming']==False]
mis_nongroom_FP_lst = list(df_mis_nongroom_FP['clip_name'])
print('mis_nongroom_FP_lst', len(mis_nongroom_FP_lst))


                                  clip  target_x  pred_x  pred_y  pred  diff
51       Grooming_930217_L47005_003910         1       0       0     0     1
218      Grooming_930218_L27280_002244         1       0       0     0     1
220      Grooming_930218_L31480_002614         1       0       0     0     1
266      Grooming_930218_L55005_004550         1       0       0     0     1
271      Grooming_930218_L57260_004743         1       0       0     0     1
...                                ...       ...     ...     ...   ...   ...
2954  Nongrooming_930219_R71320_005926         0       1       0     1     1
2955  Nongrooming_930219_R71360_005928         0       0       1     1     1
2971  Nongrooming_930219_R73735_010126         0       1       1     1     1
2987  Nongrooming_930219_R75255_010242         0       0       1     1     1
3013  Nongrooming_930219_R78565_010528         0       0       1     1     1

[465 rows x 6 columns]


,clip,target_x,pred_x,pred_y,pred,diff,clip_name,grooming
51,Grooming_930217_L47005_003910,1,0,0,0,1,930217_L47005_003910,True
218,Grooming_930218_L27280_002244,1,0,0,0,1,930218_L27280_002244,True
220,Grooming_930218_L31480_002614,1,0,0,0,1,930218_L31480_002614,True
266,Grooming_930218_L55005_004550,1,0,0,0,1,930218_L55005_004550,True
271,Grooming_930218_L57260_004743,1,0,0,0,1,930218_L57260_004743,True
...,...,...,...,...,...,...,...,...
2954,Nongrooming_930219_R71320_005926,0,1,0,1,1,930219_R71320_005926,False
2955,Nongrooming_930219_R71360_005928,0,0,1,1,1,930219_R71360_005928,False
2971,Nongrooming_930219_R73735_010126,0,1,1,1,1,930219_R73735_010126,False
2987,Nongrooming_930219_R75255_010242,0,0,1,1,1,930219_R75255_010242,False


mis_groom_FN_lst 8
mis_nongroom_FP_lst 457


In [26]:
date_lst = ['930217', '930218', '930219']
ddir = [x for x in outpath.iterdir() if x.is_dir() and x.name.split('-')[0] in date_lst]
ddir = sorted(ddir)
print(ddir)

mp4_lst = []
for dd in ddir:
    lst = sorted(dd.glob('*.mp4'))
    mp4_lst.extend(lst)
    
print(len(mp4_lst))
print(mp4_lst[0].stem.split('.', 1)[0])
mp4_mis = [x for x in mp4_lst if x.stem.split('.', 1)[0] in mis_groom_FN_lst]
print('find clip ', len(mp4_mis))
stitch_video(mp4_mis, '930217_mis_groom_FN')

mp4_mis = [x for x in mp4_lst if x.stem.split('.', 1)[0] in mis_nongroom_FP_lst]
print('find clip ', len(mp4_mis))
stitch_video(mp4_mis, '930217_mis_nongroom_FP')

[WindowsPath('d:/rat_data/new_out/930217-base-1d'), WindowsPath('d:/rat_data/new_out/930218-base-2d'), WindowsPath('d:/rat_data/new_out/930219-B-car-3-1d')]
3029
930217_L00130_000006
find clip  8


out video:  d:\rat_data/930217_mis_groom_FN.mp4
stitch_video finished
find clip  441


out video:  d:\rat_data/930217_mis_nongroom_FP.mp4
stitch_video finished
